In [2]:
import requests
import lxml.html
import pprint

from delete_database_entries import *

## Note: start your local server!

In the django project directory (where the manage.py file is located) type: "python manage.py runserver"

## Deleting all entries in the database to make a clean start

To keep the database clean we delete all content first :O

In [3]:
#delete_models("observations")
#delete_models("pipelineconfigurations")
delete_models("sessions")

Deleting: sessions
2


## We can make a session object

Here imagine that the session object is an object that will actually run your pipeline configuration on your data.

In [6]:
with requests.Session() as s:
    response = s.post('http://localhost:8000/sessions/', data={"name": "run 1 for clown nebula"})
    print("Response code: ", response.status_code)
    pp.pprint(response.json())

Response code:  201
{   'date_created': '2018-06-21T08:55:54.335291Z',
    'date_modified': '2018-06-21T08:55:54.335291Z',
    'id': 3,
    'name': 'run 1 for clown nebula'}


## Now lets have a look at the content of the database through some get requests

### Sessions:

In [7]:
response = s.get('http://localhost:8000/sessions/')
pp.pprint(response.json())

[   {   'date_created': '2018-06-21T08:55:54.335291Z',
        'date_modified': '2018-06-21T08:55:54.335291Z',
        'id': 3,
        'name': 'run 1 for clown nebula'}]
